In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

# 12.1 Categorical 데이터

In [2]:
# pandas의 Categorical을 활용하여 pandas 메모리 사용량을 줄이고 성능을 개선할 수 있는 방법 소개

* 개발 배경과 동기

In [3]:
# 하나의 컬럼 내에서 특정 값이 반복되어 존재하는 경우 다반사
# unique와 value_counts로 배열 내에서 유일한 값을 추춯하거나 특정 값이 얼마나 많이 존재하는지 확인 가능

values=pd.Series(['apple', 'orange', 'apple','apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [5]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [6]:
# 데이터웨어하우스는 분석 컴퓨팅 외 여러 다양한 데이터 시스템은 중복되는 데이터를 얼마나 효율적으로 저장하고 계산 가능한가를 중점
# 데이터웨어하우스의 경우 일반적으로 구별되는 값을 담고 있는 차원테이블과 그 테이블을 참조하는 정수키를 사용

values=pd.Series([0,1,0,0]*2)

In [7]:
dim=pd.Series(['apple','orange'])

In [8]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim

0     apple
1    orange
dtype: object

In [10]:
dim.take(values) # take메서드를 사용하면 Series 내에 저장된 원래 문자열을 구할 수 있음

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

* pandas의 Categorical

In [11]:
# pandas에는 정수 기반의 범주형 데이터를 표현(또는 인코딩)할 수 있는 Categorical형 존재

fruits=['apple','orange','apple','apple']*2

In [12]:
N=len(fruits)

In [13]:
df=pd.DataFrame({'fruit':fruits,
                'basket_id':np.arange(N),
                'count':np.random.randint(3,15,size=N),
                'weight':np.random.uniform(0,4,size=N)},
                columns=['basket_id','fruit','count','weight'])
df

,basket_id,fruit,count,weight
0,0,apple,9,2.186963
1,1,orange,10,2.585994
2,2,apple,3,1.003208
3,3,apple,7,1.300968
4,4,apple,12,2.270301
5,5,orange,14,3.646498
6,6,apple,4,0.493765
7,7,apple,11,3.512107


In [14]:
fruit_cat=df['fruit'].astype('category') # df['fruit']은 파이썬 문자열 객체의 배열, 쉽게 범주형 데이터로 변경 가능
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [15]:
c=fruit_cat.values # fruit_cat의 값은 NumPy배열이 아닌 pandas.Categorical의 인스턴스
type(c)

pandas.core.arrays.categorical.Categorical

In [16]:
c.categories # Categorical객체는 categories와 codes의 속성을 가진다.

Index(['apple', 'orange'], dtype='object')

In [17]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [18]:
df['fruit']=df['fruit'].astype('category') # 변경 완료된 값을 대입하여 DF의 컬럼을 범주형으로 변경 가능
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [19]:
# 파이썬 열거형에서 pandas.Categorical형을 직접 생성하는 것도 가능

my_categories=pd.Categorical(['foo','bar','baz','foo','bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [20]:
# 정의된 범주와 범주 코드가 있다면 from_codes 함수를 이용해서 범주형 데이터를 생성하는 것도 가능

categories=['foo','bar','baz']

In [21]:
codes=[0,1,2,0,0,1]

In [22]:
my_cats_2=pd.Categorical.from_codes(codes,categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [23]:
ordered_cat=pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [24]:
my_cats_2.as_ordered() # 순서가 없는 범주형 데이터는 as_ordered메서드를 이용해 순서를 가지도록 만들 수 있다.

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

* Categorical 연산

In [25]:
# padans에서 Categorical은 문자열 배열처럼 인코딩되지 않은 자료형을 사용하는 방식과 거의 유사
# 예) 임의의 숫자 데이터 pandas.qcut함수로 구분해보면 pandas.Categorical객체를 반환

np.random.seed(12345)

In [26]:
draws=np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [27]:
bins=pd.qcut(draws,4) # 사분위로 나누고 통계내기
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [28]:
bins=pd.qcut(draws, 4, labels=['Q1','Q2','Q3','Q4']) # labels인자로 직접 이름 지정
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [29]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [30]:
bins=pd.Series(bins, name='quartile')

In [31]:
results=(pd.Series(draws) # groupby를 이용해서 요약 통계 내기
        .groupby(bins)
        .agg(['count', 'min', 'max'])
        .reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [32]:
results['quartile'] # 결과에서 quartile컬럼은 bins의 순서를 포함한 원래 범주 정보 유지

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [33]:
# categorical을 이용한 성능 개선: 특정 데이터셋에 대해 다양한 분석을 하는 경우 범주형으로 변환하는 것만으로도 개선
# 예) 소수의 독립적인 카테고리로 분류되는 천만 개의 값을 포함하는 Series를 살펴보기

N=10000000

In [34]:
draws=pd.Series(np.random.randn(N))

In [36]:
labels=pd.Series(['foo','bar','baz','qux']*(N//4))

In [37]:
categories=labels.astype('category') # labels를 categorical로 변환

In [38]:
labels.memory_usage() # categories가 labels에 비해 더 적은 메모리를 사용하는 것을 확인

80000128

In [39]:
categories.memory_usage()

10000320

In [40]:
%time _ =labels.astype('category') # 범주형으로 변환하는 과정은 한 번만 변환하면 되는 일회성 비용

Wall time: 1.26 s


* Categorical 메서드

In [41]:
# 범주형 데이터를 담고 있는 Series는 특화된 문자열 메서드는 Series.str과 유사한 몇가지 특수 메서드 제공
# 이를 통해 categories와 codes에 쉽게 접근 가능

s=pd.Series(['a','b','c','d']*2)

In [42]:
cat_s=s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [43]:
cat_s.cat.codes # 특별한 속성 cat을 이용해 categorical메서드에 접근 가능

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [44]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [46]:
actual_categories=['a','b','c','d','e'] # 4종류가 넘는 것을 실제 카테고리가 이미 알고 있다면 set_categories메서드로 변경 가능

In [47]:
cat_s2=cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [48]:
cat_s.value_counts() 

d    2
c    2
b    2
a    2
dtype: int64

In [49]:
cat_s2.value_counts() # value_counts를 호출해보면 변경된 카테고리 반영(새로운 카테고리 추가)

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [50]:
# 큰 데이터셋을 다룰 경우 categorical을 이용하면 메모리 절약, 성능 개선 가능
# remove_unused_categories메서드로 DF나 Series를 걸러낸 후 실제로 데이터에 존재하지 않는 카테고리를 제거

cat_s3=cat_s[cat_s.isin(['a','b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [51]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

In [52]:
# 모델링을 위한 더미값 생성: 범주형 데이터를 더미값으로 변환(원핫 인코딩)
# 이를 위해 각각 구별되는 카테고리를 컬럼으로 가지는 DF생성, 각 컬럼에는 해당 카테고리 여부에 따라 0과 1의 값을 갖게 됨

cat_s=pd.Series(['a','b','c','d']*2, dtype='category')

In [53]:
pd.get_dummies(cat_s) # pandas.get_dummies함수는 이런 1차원 범주형 데이터를 더미값으로 가지는 DF로 변환

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


# 12.2 고급 GroupBy 사용

* 그룹 변환과 GroupBy 객체 풀어내기

In [54]:
# transform이라는 내장 메서드를 이용하면 apply메서드와 유사하게 동작하면서 사용할 수 있는 함수의 종류에 대해 더 많은 제한 포함
# 1.그룹형태로 브로드캐스트할 수 있는 스칼라 값 생성 2.입력 그룹과 같은 형태의 객체 반환 3.입력 변경 불가

df=pd.DataFrame({'key':['a','b','c']*4,
                'value':np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [55]:
g=df.groupby('key').value
g.mean() # key에 따른 그룹의 평균 구하기

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [56]:
g.transform(lambda x: x.mean()) # Series형태가 아닌 값을 변경하기 원할 경우 transform에 람다 함수 넘기기

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [57]:
g.transform('mean') # 내장 요약함수에 대해서는 agg메서드에서처럼 문자열 그룹 연산 이름 넘기기

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [58]:
g.transform(lambda x: x*2) # transform은 Series를 반환하는 함수만 사용 가능, 하지만 결과는 입력과 똑같은 크기

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [59]:
g.transform(lambda x: x.rank(ascending=False)) # 내림차순 순위 계산

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [60]:
def normalize(x):
    return (x-x.mean()/x.std())

In [61]:
g.transform(normalize)

0    -1.161895
1    -0.420094
2     0.321707
3     1.838105
4     2.579906
5     3.321707
6     4.838105
7     5.579906
8     6.321707
9     7.838105
10    8.579906
11    9.321707
Name: value, dtype: float64

In [62]:
g.apply(normalize)

0    -1.161895
1    -0.420094
2     0.321707
3     1.838105
4     2.579906
5     3.321707
6     4.838105
7     5.579906
8     6.321707
9     7.838105
10    8.579906
11    9.321707
Name: value, dtype: float64

In [63]:
g.transform('sum') # 내장 요약함수는 transform이 더 빠르게 동작, 뒤로 되돌리기 가능(그룹 연산 풀어낼 수 있음)

0     18.0
1     22.0
2     26.0
3     18.0
4     22.0
5     26.0
6     18.0
7     22.0
8     26.0
9     18.0
10    22.0
11    26.0
Name: value, dtype: float64

In [64]:
normalized=(df['value']-g.transform('mean'))/g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

* 시계열 그룹 리샘플링

In [65]:
# 시계열 데이터에서 resample메서드는 의미적으로 시간 간격에 기반한 그룹 연산

N=15

In [67]:
times=pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

In [68]:
df=pd.DataFrame({'time':times,
                'value':np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [69]:
df.set_index('time').resample('5min').count() # time으로 색인 후 리샘플

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [70]:
df2=pd.DataFrame({'time':times.repeat(3), # key컬럼으로 구분되는 여러 시계열 데이터를 담고 있는 DataFrame
                 'key': np.tile(['a','b','c'],N),
                 'value': np.arange(N*3.)})
df[:7]

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6


In [76]:
time_key=pd.Grouper(freq='5min') # key의 각 값에 대해 같은 리샘플을 수행하기 위해 TimeGrouper이용

In [77]:
resampled=(df2.set_index('time')
          .groupby(['key',time_key])
          .sum())
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

# 12.3 메서드 연결 기법

In [ ]:
# 데이터셋을 여러 차례 변형해야 하는 경우 분석에는 전혀 필요 없는 임시 변수를 계속 생성해야하는 상황 발생

df=load_data()
df2=df[df['col2']<0]
df2['col_demeaned']=df2['col1']-df2['col1'].mean()
result=df2.groupby('key').col1_demeand.std()

In [ ]:
# 실용적이지 않은 방법
df2=df.copy()
df2['k']=v

# 실용적인 방법
df2=df.assign(k=v) # 객체를 변경하는 대신 값 대입이 완료된 새로운 DataFrame 반환, assign을 사용하면 메서드 연결 가능

In [ ]:
# 메서드 연결 시 주의할 점은 임시 객체를 참조해야 할 경우가 있을 수 있다.
# load_data의 반환값을 임시 변수인 df에 담기 전까지는 그 결과를 참조 불가
# 이런 경우 assign이나 호출이 가능한 객체 또는 함수를 인자로 받는 pandas의 다른 함수로 해결

result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())
         .groupby('key')
         .col1_demeaned.std())

In [ ]:
# 호출이 가능한 객체의 예시- 위 예제의 일부 코드
df=load_data()
df2=df[df['col2']<0]

# 다음과 같이 변경 가능
df=(load_data()
   [lambda x: x['col2']<0])

In [ ]:
# 전체 코드를 하나의 메서드 연결 표현으로 작성 가능
result(load_data()
      [lambda x: x.col2<0]
      .assign(col1_demeaned=lambda x: x.col1-x.col1.mean())
      .groupby('key')
      .col1_demeaned.std())

* pipe 메서드

In [ ]:
# pipe메서드로 메서드 연결에서 직접 작성한 함수나 다른 서드파티 라이브러리 함수를 사용

a=f(df, arg1=v1)
b=g(a, v2, arg3=v3)
c=h(b,arg4=v4)

In [ ]:
# f(df)와 df.pipr(f)는 동일
result = (df.pipe(f, arg1=v1)
         .pipe(g,v2,arg3=v3)
         .pipe(h, arg4=v4))

In [ ]:
# pipe를 이용한 패턴 중 하나는 일련의 연산 재사용 가능 함수로 일반화하는 것 

g=df.groupby(['key1','key2'])
df['col1']=df['col1']-g.transform('mean') # 컬럼에서 그룹 평균을 빼는 과정

In [ ]:
# 한 컬럼이 아니라 여러 컬럼에 대해 그룹 평균을 뺄 수 있고 그룹의 키를 쉽게 변경할 수 있길 바란다면

def group_dmean(df, by, cols):
    result=df.copy()
    g=df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result

In [ ]:
result = (df[df.col1<0]
         .pipe(group_dmean, ['key1','key2'],['col1']))